# 5-bus Market simulation with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways. This
example demonstrates some of these capabilities to represent electricity market clearing.

## Dependencies and Data
First, let's create `System`s to represent the Day-Ahead and Real-Time market clearing
process with hourly, and 5-minute time series data, respectively.

### Modeling Packages

In [1]:
using PowerSystems
using PowerSimulations
using PowerSystemCaseBuilder

### Data management packages

In [2]:
using Dates
using DataFrames

### Optimization packages

In [3]:
using Cbc # mip solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)
using Ipopt # solver that supports duals
ipopt_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)

MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("print_level") => 0])

### 5-bus Data
The five bus system data here includes hourly day-ahead data, 5-minute real-time market
data, and 6-second actual data. We'll only use the hourly and 5-minute data for the
example simulations below, but the 6-second data is included for future development.

In [4]:
sys_DA = build_system(SIIPExampleSystems, "5_bus_matpower_DA")
sys_RT = build_system(SIIPExampleSystems, "5_bus_matpower_RT")

┌ Info: Building new system 5_bus_matpower_DA from raw data
└   sys_descriptor.raw_data = "/Users/cbarrows/.julia/packages/PowerSystemCaseBuilder/1xViZ/data/matpower/case5_re_uc.m"
[ Info: extending matpower format with data: areas 1x3
[ Info: extending matpower format with data: gen_name 7x4
[ Info: extending matpower format by appending matrix "gen_name" in to "gen"
[ Info: reversing the orientation of branch 6 (4, 3) to be consistent with other parallel branches
[ Info: the voltage setpoint on generator 4 does not match the value at bus 4
[ Info: the voltage setpoint on generator 1 does not match the value at bus 1
[ Info: the voltage setpoint on generator 5 does not match the value at bus 10
[ Info: the voltage setpoint on generator 2 does not match the value at bus 1
[ Info: the voltage setpoint on generator 3 does not match the value at bus 3
[ Info: removing 1 cost terms from generator 4: [4000.0, 0.0]
[ Info: removing 1 cost terms from generator 1: [1400.0, 0.0]
[ Info: removin

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,6
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
4,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
5,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,PhaseShiftingTransformer,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
7,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
8,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
9,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5


## `OperationsProblemTemplate`s

In [5]:
template_uc = template_unit_commitment()
template_ed = template_economic_dispatch()


Operations Problem Specification
Transmission: CopperPlatePowerModel
Devices Models: 

	Type: ThermalStandard
 	Formulation: ThermalDispatch

	Type: HydroDispatch
 	Formulation: HydroDispatchRunOfRiver

	Type: PowerLoad
 	Formulation: StaticPowerLoad

	Type: RenewableFix
 	Formulation: FixedOutput

	Type: RenewableDispatch
 	Formulation: RenewableFullDispatch

	Type: HydroEnergyReservoir
 	Formulation: HydroDispatchRunOfRiver

	Type: InterruptibleLoad
 	Formulation: InterruptiblePowerLoad

Branches Models: 

	Type: Line
 	Formulation: StaticBranch

	Type: TapTransformer
 	Formulation: StaticBranch

	Type: Transformer2W
 	Formulation: StaticBranch

	Type: HVDCLine
 	Formulation: HVDCDispatch

Services Models:

	Type: VariableReserve{ReserveDown}
 	Formulation: RangeReserve

	Type: VariableReserve{ReserveUp}
 	Formulation: RangeReserve



### Define the Simulation Sequence

In [6]:
problems = SimulationProblems(
    UC = OperationsProblem(
        template_uc,
        sys_DA,
        optimizer = solver,
        balance_slack_variables = true,
    ),
    ED = OperationsProblem(
        template_ed,
        sys_RT,
        optimizer = ipopt_solver,
        constraint_duals = [:CopperPlateBalance],
    ),
)

feedforward_chronologies = Dict(("UC" => "ED") => Synchronize(periods = 24))

feedforward = Dict(
    ("ED", :devices, :ThermalStandard) => SemiContinuousFF(
        binary_source_problem = ON,
        affected_variables = [ACTIVE_POWER],
    ),
)

#cache = Dict("UC" => [TimeStatusChange(ThermalStandard, PSI.ON)])
intervals = Dict("UC" => (Hour(24), Consecutive()), "ED" => (Hour(1), Consecutive()))

DA_RT_sequence = SimulationSequence(
    problems = problems,
    intervals = intervals,
    ini_cond_chronology = InterProblemChronology(),
    feedforward_chronologies = feedforward_chronologies,
    feedforward = feedforward,
)

Feed Forward Chronology
-----------------------

ED: SemiContinuousFF -> ThermalStandard

                     UC--┐ from : On
                         |
┌----┬----┬----┬----┬----┼----┬----┬----┬----┬----┬----┐
|    |    |    |    |    |    |    |    |    |    |    |
|    |    |    |    |    |    |    |    |    |    |    |
└─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED ... (x24) to : ["P"]

Initial Condition Chronology
----------------------------

1
|
|
2 --> 2 ... (x24)   


## `Simulation`

In [7]:
file_path = mkpath(joinpath(".", "5-bus-simulation"))
sim = Simulation(
    name = "5bus-test",
    steps = 1,
    problems = problems,
    sequence = DA_RT_sequence,
    simulation_folder = file_path,
)

Simulation()


### Build simulation

In [8]:
build!(sim)

┌ Info: 
│  ──────────────────────────────────────────────────────────────────────────────
│                                        Time                   Allocations      
│                                ──────────────────────   ───────────────────────
│        Tot / % measured:            1.78s / 100%             148MiB / 100%     
│ 
│  Section               ncalls     time   %tot     avg     alloc   %tot      avg
│  ──────────────────────────────────────────────────────────────────────────────
│  Build Simulation           1    1.78s   100%   1.78s    148MiB  100%    148MiB
│    Build Problems           1    1.77s   100%   1.77s    148MiB  100%    148MiB
│      Problem UC             1    1.21s  68.3%   1.21s    117MiB  79.2%   117MiB
│        ThermalStandard      1    521ms  29.3%   521ms   41.0MiB  27.7%  41.0MiB
│        Services             1   96.8ms  5.45%  96.8ms   10.5MiB  7.06%  10.5MiB
│        Build pre-step       1   29.4ms  1.65%  29.4ms   2.79MiB  1.89%  2.79MiB
│   

BuildStatus.BUILT = 0

### Execute simulation
```julia

In [9]:
execute!(sim, enable_progress_bar = false)

Presolve 701 (-2599) rows, 1589 (-5971) columns and 2930 (-11098) elements
Perturbing problem by 0.001% of 10 - largest nonzero change 0.00010943053 ( 54.949531%) - largest zero change 0.00010938892
0  Obj 0.0082926214 Primal inf 370.67927 (425)
89  Obj 13060.303 Primal inf 182.89395 (301)
178  Obj 24299.651 Primal inf 100.37047 (232)
267  Obj 30950.67 Primal inf 56.412648 (179)
356  Obj 31158.922 Primal inf 23.524935 (120)
445  Obj 31571.851 Primal inf 5.3320629 (47)
498  Obj 31907.354
Optimal - objective value 31907.325
After Postsolve, objective 31907.325, infeasibilities - dual 42318.909 (240), primal 0 (0)
Presolved model was optimal, full model needs cleaning up
0  Obj 31907.325
Optimal - objective value 31907.325
Optimal objective 31907.32482 - 498 iterations time 0.012, Presolve 0.01

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code

RunStatus.SUCCESSFUL = 0

```

In [10]:
# Results

First we can load the result metadata
```julia
results = SimulationResults(sim);
uc_results = get_problem_results(results, "UC")
ed_results = get_problem_results(results, "ED");
```

Then we can read and examine the results of interest
```julia
prices = read_dual(ed_results, :CopperPlateBalance)
```

or if we want to look at the realized values
```julia
read_realized_duals(ed_results)[:CopperPlateBalance]
```

*note that in this simulation the prices are all equal to the balance slack
penalty value of $100000/MWh because there is unserved energy in the result*

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*